In [ ]:
%pip install google-genai

In [ ]:
from google import genai
from google.genai import types
import time
import os

In [ ]:
api_key = "<Gemini API Key>"
# api_key = os.getenv("GOOGLE_API_KEY")

# if not api_key:
#     raise ValueError("GOOGLE_API_KEY not found in .env file") 

client = genai.Client(api_key=api_key)

In [ ]:
# Create the File Search store with an optional display name
file_search_store = client.file_search_stores.create(config={'display_name': 'generative_ai_leader'})
print(file_search_store.name)

In [ ]:
def upload_file_to_file_search_store(file_search_store_name: str, file: str, display_name: str):
    # Upload to file search store with custom chunking config
    operation = client.file_search_stores.upload_to_file_search_store(
        file_search_store_name=file_search_store_name,
        file=file,
        config={
            "display_name": display_name,
            "chunking_config": {
                "white_space_config": {
                    "max_tokens_per_chunk": 300,
                    "max_overlap_tokens": 50
                }
            }
        }
    )

    # Wait until the upload is complete
    while not operation.done:
        time.sleep(5)
        operation = client.operations.get(operation)

In [ ]:
from typing import Any

def get_citations(grounding_metadata: Any):
    if grounding_metadata is None:
        return []
    
    chunks = grounding_metadata.grounding_chunks
    supports = grounding_metadata.grounding_supports

    has_grounding = chunks is not None and supports is not None
    print (has_grounding)
    if has_grounding is False:
        return []

    citations = []
    for support in supports:
        indices = support.grounding_chunk_indices
        for index in indices:
            source = chunks[index].retrieved_context
            source_title = source.title
            citation = support.segment.text
            start_index = support.segment.start_index
            end_index = support.segment.end_index

            citation = {
                "title": source_title,
                "text": f"[{start_index}-{end_index}] {citation}"
            }
            citations.append(citation)

    return citations

In [ ]:
def generate_answer(question: str):    
    # Ask a question about the file
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=question,
        config=types.GenerateContentConfig(tools=[
            types.Tool(file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name],
                top_k=1
            ))
        ])
    )

    citations = get_citations(response.candidates[0].grounding_metadata) if response is not None and response.candidates is not None and len(response.candidates) > 0 and response.candidates[0].grounding_metadata is not None else []

    return {
        "answer": response.text,
        "citations": citations
    }

In [ ]:
upload_file_to_file_search_store(
    file_search_store_name=file_search_store.name, 
    file="generative_ai_leader_study_guide_english.pdf", 
    display_name="genai_leader_study_guide")

In [ ]:
upload_file_to_file_search_store(
    file_search_store_name=file_search_store.name, 
    file="generative_ai_leader_exam_guide_english.pdf", 
    display_name="genai_leader_exam_guide")

In [ ]:
response = generate_answer(question="""How to make a successful Generative AI solution?""")

print("Answer", response["answer"])

In [ ]:
for citation in (response["citations"] or []):
    print(f"Source: {citation["title"]}, text: {citation["text"]}")

In [ ]:
response = generate_answer(question="""What will be tested in section 3 of the exam guide? Please output in Markdown format""")

print("Answer", response["answer"])

In [ ]:
for citation in (response["citations"] or []):
    print(f"Source: {citation["title"]}, text: {citation["text"]}")

In [ ]:
response = generate_answer(question="""What is the capital of China?""")

print("Answer", response["answer"])
for citation in (response["citations"] or []):
    print(f"Source: {citation["title"]}, text: {citation["text"]}")

In [ ]:
# Check the file search store is deleted permanently
all_stores = client.file_search_stores.list()
for store in all_stores:
        # Delete a file search store
        client.file_search_stores.delete(name=store.name, config={ "force": True })
        print(store.name, " is deleted.")
